In [3]:
import pyscf
import pyscf.tools
import sys

from orbitalpartitioning import *

In [7]:
molecule = '''
He       0.0000000000      0.0000000000      0.0000000000                 
He       0.0000000000      2.0000000000      0.0000000000                 
He       2.0000000000      0.0000000000      0.0000000000                 
He       2.0000000000      2.0000000000      0.0000000000                 
He       0.0000000000      0.0000000000      5.0000000000                 
He       0.0000000000      2.0000000000      5.0000000000                 
He       2.0000000000      0.0000000000      5.0000000000                 
He       2.0000000000      2.0000000000      5.0000000000 
'''
basis = "6-31g*"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        #symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
#print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
#mf.conv_tol_grad = 1e-5
#mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
print("size of MO coeff", C.shape)
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing
print(ndocc)
print(nsing)
print(nvirt)
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]
np.save("/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras", C)

nbas = Cdocc.shape[0]



******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/bd/dlq8bnjd2fsb6s4vr5njxgp40000gn/T/tmppt4h3urs
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 8  num. singly occ = 0
Set gradient conv threshold to 0.0001
init E= -22.8657603629798
  HOMO = -0.846668588238936  LUMO = 1.20583461726369
cycle= 1 E= -22.8247505899441  delta_E= 0.041  |g|= 0.0113  |ddm|=  0.2
  HOMO = -0.870885382864021  LUMO = 1.20883405453442
cycle= 2 E= -22.8247816130477  delta_E= -3.1e-05  |g|= 0.00126  |ddm|= 0.00619
  HOMO = -0.87079077616662  LUMO = 1.20891940507576
cycle= 3 E= -22.8247820062166  delta_E= -3.93e-07  |g|= 1.86e-06  |ddm|= 0.000777
  HOMO = -0.870790364387975  LUMO = 1.20891957453167
cycle= 4 

In [8]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    #print(ao_idx)
    if ao[0] in (0,1,2,3):
        frag1.append(ao_idx)
        full.append(ao_idx)
    if ao[0] in (4,5,6,7):
        frag2.append(ao_idx)
        full.append(ao_idx)  


frags = [frag1, frag2]
print(frags)
print(len(frag1))
print(len(frag2))

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15]]
8
8


In [9]:
# Define projectors
nbas = Cdocc.shape[0]
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])
display(Pfull.shape)

(16, 16)

In [10]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))
display(Cact)
pyscf.tools.molden.from_mo(mf.mol, "/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/Cact.molden", Cact)
print(" Should be 1: ", np.linalg.det(Cact.T @ S @ Cact))

 Partition   16 orbitals into a total of   16 orbitals
            Index   Sing. Val. Space       
                0   1.00000000            0*
                1   1.00000000            2*
                2   1.00000000            2*
                3   1.00000000            0*
                4   1.00000000            0*
                5   1.00000000            2*
                6   1.00000000            0*
                7   1.00000000            2*
                8   1.00000000            2*
                9   1.00000000            2*
               10   1.00000000            2*
               11   1.00000000            0*
               12   1.00000000            0*
               13   1.00000000            0*
               14   1.00000000            2*
               15   1.00000000            0*
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
              0              0              8
              1              0          

array([[-2.86492473e-01, -1.65456563e-02,  7.58920221e-02,
         1.96552549e-01, -3.81425212e-01, -1.53824796e-02,
        -2.01469513e-01,  1.97087785e-01, -6.19112396e-01,
         3.18257164e-01, -2.50053814e-01,  3.59226123e-01,
        -2.20025637e-02, -3.94105916e-01, -1.52548068e-01,
        -6.99535798e-01],
       [-2.45661153e-01, -1.44491737e-02,  6.74757245e-02,
         1.70631343e-01, -3.33088538e-01, -1.50211776e-02,
        -1.72970970e-01,  1.72272984e-01,  7.04574897e-01,
        -4.01030928e-01,  2.56404616e-01, -3.97778473e-01,
         1.57813888e-02,  3.55735581e-01,  1.59893697e-01,
         6.81288265e-01],
       [-1.98626447e-01,  2.57241180e-01, -1.89118404e-01,
         2.26056729e-01,  3.38865973e-01, -4.86691946e-02,
        -2.01469513e-01, -4.02226781e-02,  7.91391894e-01,
        -8.69653728e-03, -4.48100176e-01, -2.67100322e-01,
         9.33416032e-02, -3.94105916e-01,  2.89954982e-01,
        -4.44931232e-01],
       [-1.73190230e-01,  2.21360153e

 Should be 1:  0.9999999999999973


In [11]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    # (Of, Sf, Vf), (Oact, Sact, Vact) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)

Cact = np.hstack(Cfrags)

print("nick: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
#pyscf.tools.molden.from_mo(mf.mol, "Pfull_no.molden", Pfull)
#pyscf.tools.molden.from_mo(mf.mol, "Cact_no.molden", Cact)
#pyscf.tools.molden.from_mo(mf.mol, "Cenv_no.molden", Cenv)
pyscf.tools.molden.from_mo(mf.mol, "/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/Cfrag_sym_pc%i.molden"%i, Cfrags[i])
#    #pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])


print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 Fragment:  [0, 1, 2, 3, 4, 5, 6, 7]
 Partition   16 orbitals into a total of    8 orbitals
            Index   Sing. Val. Space       
                0   1.00000000            0*
                1   1.00000000            0*
                2   1.00000000            0*
                3   1.00000000            0*
                4   1.00000000            2*
                5   1.00000000            2*
                6   1.00000000            2*
                7   1.00000000            2*
                8   0.00000248            0
                9   0.00000229            0
               10   0.00000229            0
               11   0.00000212            0
               12   0.00000145            2
               13   0.00000127            2
               14   0.00000127            2
               15   0.00000111            2
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
              0              4              4
          

In [13]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/ints_h0", h0)
np.save("/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/ints_h1", h1)
np.save("/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/ints_h2", h2)
np.save("/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/mo_coeffs", Cact)
np.save("/Users/ayush/workspace/cmf/project_hessian_debug/ClusterMeanField.jl/examples/ras_two_benzene/he_ras/overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
#np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
#np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

(16, 0)
(16, 16)
(16, 16)
